In [28]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git 
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install bitsandbytes // valid
# !pip install transformers
# !pip install peft accelerate
# !pip install -q datasets
# !pip install --upgrade pyarrow
# !pip install pyarrow==15.0.2
# model_id = "EleutherAI/gpt-neo-1.3B"
# !pip install --upgrade pyarrow
# !pip install datasets==2.12.0 transformers==4.28.0
# !pip install peft accelerate
# !pip install -q datasets
# !pip install transformers[torch]
# !pip install torch --upgrade
# !pip install transformers --upgrade
# !pip install pyarrow==5.0.0
# !pip install tensorflow
# !pip install tf-keras
# !pip install peft
# !pip install transformers
# !pip install wandb
# !pip install transformers[torch]
# !pip install torch==2.0.0+cu118 torchvision==0.15.0+cu118 torchaudio==2.0.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html
model_id = "gpt2-large"
model_name = "gpt2-large"

import wandb
import os
os.environ["WANDB_PROJECT"]=model_name
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="false"
os.environ["WANDB_NAME"]="first"
os.environ["WANDB_API_KEY"] = "b689f7c91f1ec7520fa8da927f175f1efd587181"
# wandb.login(key="b689f7c91f1ec7520fa8da927f175f1efd587181")
import torch
# import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel, TextDataset, DataCollatorForLanguageModeling, TrainingArguments, GPT2LMHeadModel ## do poprawki
from transformers import Trainer
# from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
# from datasets import load_dataset
import gc
import torch
# import pyarrow.parquet as pq


default_path = 'Magisterka'
# data_path_output = os.path.join(default_path,"EleutherAI", model_name)
data_path_output = os.path.join(default_path, model_name)


In [29]:
# !pip install datasets --upgrade
import os
from transformers import AutoTokenizer, GPTNeoForCausalLM
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader
import pandas as pd
# model_name = model_id
frac_value = 1
# Load a pre-trained tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(data_path_output)
except:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.bos_token = "<|startoftext|>"
tokenizer.save_pretrained(data_path_output)

path_wow_quests = os.path.join("data.csv")
path_wow_quests_save_train = os.path.join("data_train.txt")
path_wow_quests_save_validation = os.path.join("data_validation.txt")

text_data_train = []
text_data_valid = []
data = pd.read_csv(path_wow_quests)
data = data.dropna()
data = data.dropna(subset=['Quest Description'])
data = data.sample(frac=frac_value, random_state=42).reset_index(drop=True)
train_ratio = 0.9
eval_ratio = 1 - train_ratio

num_train_samples = int(train_ratio * len(data))
num_eval_samples = len(data) - num_train_samples
train_data = data.iloc[:num_train_samples]
eval_data = data.iloc[num_train_samples:]

eos_token = tokenizer.eos_token
sos_token = tokenizer.bos_token
for i, row in train_data.iterrows():
    descrp_org = str(row["Quest Description"])
    data_to_save = sos_token + "Title: " + str(row["Quest Title"]) + " Description: " + descrp_org + eos_token
    data_to_save = data_to_save.replace('\n', '')
    text_data_train.append(data_to_save)

for i, row in eval_data.iterrows():
    descrp_org = str(row["Quest Description"])
    data_to_save = sos_token + "Title: " + str(row["Quest Title"]) + " Description: " + descrp_org + eos_token
    data_to_save = data_to_save.replace('\n', '')
    text_data_valid.append(data_to_save)

with open(path_wow_quests_save_train, "w", newline="\n") as file:
    for text in text_data_train:
      file.write(text)

with open(path_wow_quests_save_validation, "w", newline="\n") as file:
    for text in text_data_valid:
      file.write(text)

def load_text_data(data):
    return pd.DataFrame(data, columns=["text"])


train_df = load_text_data(text_data_train)
validation_df = load_text_data(text_data_valid)

# print(train_df["text"])
# Convert the pandas DataFrame to a Hugging Face Dataset
# train_dataset = Dataset.from_pandas(train_df)
# validation_dataset = Dataset.from_pandas(validation_df)

# # Define a tokenization function
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)


# def find_tokenizer_length():
#     max_tokenizer_length = 0
#     eos_token_id = 50256
#     for row in train_dataset:
#         print(row)
#         tokens = tokenize_function(row).count(eos_token_id)
#         if tokens > max_tokenizer_length:
#             max_tokenizer_length = tokens
#     for row in validation_dataset:
#         tokens = tokenize_function(row).count(eos_token_id)
#         if tokens > max_tokenizer_length:
#             max_tokenizer_length = tokens
#     print(max_tokenizer_length)
    
# find_tokenizer_length()

# Apply the tokenization function to the datasets
# train_dataset = train_dataset.map(tokenize_function, batched=True)
# validation_dataset = validation_dataset.map(tokenize_function, batched=True)

# train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
# validation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

train_dataset = TextDataset(tokenizer=tokenizer, file_path=path_wow_quests_save_train, block_size=64)
validation_dataset = TextDataset(tokenizer=tokenizer, file_path=path_wow_quests_save_validation, block_size=64)
# train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# validation_dataloader = DataLoader(validation_dataset, batch_size=8, shuffle=False)


# tokenizer(batch_sentences, padding='max_length', truncation=True)

print(train_dataset)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [33]:
# train_dataset = TextDataset(tokenizer=tokenizer, file_path="data_train.txt", block_size=64)
# validation_dataset = TextDataset(tokenizer=tokenizer, file_path="data_validation.txt", block_size=64)
from torch.utils.data import DataLoader
from accelerate import Accelerator
from transformers import EarlyStoppingCallback

def fine_tune_gpt2(model_id, output_dir):
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    else:
        device = torch.device("cpu")
        print("CUDA is not available. Using CPU.")
    model = None
    gc.collect()
    try:
        torch.cuda.empty_cache()
    except RuntimeError as e:
        print(f"Caught a runtime error: {e}")
        
    try:
        model = GPT2LMHeadModel.from_pretrained(output_dir) #GPT2LMHeadModel||GPTNeoForCausalLM
    except:
        model = GPT2LMHeadModel.from_pretrained(model_id) #model_id
    # print(device)
    # model = model.to("cpu")
    # train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True, pin_memory=True)
    # eval_dataloader = DataLoader(validation_dataset, batch_size=12, shuffle=False, pin_memory=True)
    # accelerator = Accelerator()
    # model, train_dataloader, eval_dataloader = accelerator.prepare(
    #     model, train_dataloader, eval_dataloader
    # )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    # Set training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=2,
        report_to="wandb",
        # eval_strategy='epoch',
        evaluation_strategy='epoch',
        save_strategy="epoch",
        per_device_train_batch_size=24, #8/24
        per_device_eval_batch_size=24, #8/24
        logging_dir= os.path.join(data_path_output, '/logs'),
        logging_steps=100,
        save_total_limit=2,
        load_best_model_at_end=True
    )
    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # stop after 3 non-improving epochs
    )
    trainer.train()
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    
    # Token indices sequence length is longer than the specified maximum sequence length for this model (244752 > 1024). Running this sequence through the model will result in indexing errors


In [34]:
 # Make only GPU 0 visible
fine_tune_gpt2(model_id, data_path_output)
# wandb.finish()

CUDA is available. Using GPU: NVIDIA L4


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.739000,2.843668
2,2.165300,2.907673


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [ ]:
#zrobić skrypt do odpalania modeli, generowania i porównywania z datasetem - zapisywane do pliku
# ocenic ktory sposob jest lepszy na trenowanie modelu (szybkosc i jakosc)
# odpalic model z kwantyzacją
# trenowac model z kwantyzacja
# znalezc nowe 2 zbiory danych
# wymyslec i zbudowac logike generowania i walidowania questow (najlepiej w oparciu o jakies zewnetrzne api wiekszego modelu) np. llama2/3

#1. 
    # odpalić na lepszym gpu..
    # sprawdzić odpalanie na kilku gpu (połączyć im memory)
#2.
    # odpalić model z kwantyzacją
#3.
    # wygenerować prompt z gpt4o lub gemini odnośnie świata WoW
        # zobaczyć api do gemini lub llama lub innego modelu
        # odpalić model neo i w nim wklepać prompt
#4. # zrobić kod do modelu 
    # odpalamy wytrenowany model
    # generujemy questy
    # (sprawdzamy, czy jest już taki quest)
    # sprawdzamy drugim modelem czy quest jest git
    # zadajemy pytania do tego questa
    # odpowiadamy na nie i sprawdzamy czy na pewno jest git

In [ ]:
# zrobić notatnik z gpt-neo..
# zrobić notatnik z modelem kwantyzowanym
# spróbować zapytania z api
# zrobić vector database..
# znalezc 2 dodatkowe zbiory danych
# wydzielic pliki do tworzenia danych i trenowania

In [8]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")
print(f"Device count: {torch.cuda.device_count()}")

# PyTorch version: 2.0.0+cu118
# CUDA available: True
# CUDA version: 11.8
# cuDNN version: 8900
# Device count: 1



PyTorch version: 2.3.1+cu121
CUDA available: True
CUDA version: 12.1
cuDNN version: 8902
Device count: 1


In [8]:
# !pip list
input_tensor = torch.randn(5, 10).to("cuda")

In [14]:
from transformers import AdamW
from tqdm import tqdm
try:
    model = GPT2LMHeadModel.from_pretrained(output_dir) #GPT2LMHeadModel||GPTNeoForCausalLM
except:
    model = GPT2LMHeadModel.from_pretrained(model_id) #model_id
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        inputs, labels = batch  # Your dataset should return tokenized inputs and labels
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average Training Loss: {avg_train_loss}")

    # Validation loop
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(validation_dataloader, desc="Validation"):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(validation_dataloader)
    print(f"Average Validation Loss: {avg_val_loss}")


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1:   0%|          | 0/1233 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)